# Overview of the use case

---------(Overview on the use case,
why is it important for data producers,
usefulness of the library
when reading non-SDMX data
and convert to SDMX. Mention validations and calculations over VTL.)---------

Talking points and agenda:

- General use of pysdmx on Data Producers
- Outside the SDMX garden, looking at LEI and GLEIF
- Data cleaning and set up using pandas
- Downloading and reading the ConceptScheme on SDMX-ML 2.1 using read_sdmx
- Retrieving the Schema from FMR (FusionJSON)
- Convert the Schema to a VTL DataStructure
- Using VTL to validate the data
- Using VTL to perform calculations
- Generate SDMX-ML file with the aggregated data
- Reading back the SDMX file using read_sdmx

### List of pysdmx classes and functions used in this notebook:

Functions:
- pysdmx.io.read_sdmx
- pysdmx.io.csv.sdmx20.writer.write

Classes:
- pysdmx.api.fmr.RegistryClient (and methods)
- pysdmx.model.message.Message (and methods)
- pysdmx.io.pd.PandasDataset
- pysdmx.model.dataflow.Schema

# Outside the SDMX garden, looking at LEI and GLEIF

---------(Explanation on LEI and GLEIF, use for data producers)---------

## Data cleaning and set up using pandas and pysdmx

For this use case,
we will use the Golden Copy file from GLEIF (link)
and filter on those LEI with status Active. 

The code renames the columns and select the data we need for later validation.
We added the possibility of saving the data into plain CSV or SDMX-CSV 2.0
(using pysdmx).

The code uses the chunking capabilities of Pandas for better memory efficiency.
This is a prototype of the streaming capabilities with pandas in pysdmx,
which will be available by the end of 2025.

For the sake of this example,
we distinguish between the Golden Copy original path
(link to download the file) and the Golden Copy Changed,
which would be the output of this code.

This code requires to install the extra data from pysdmx,
which simply install pandas.

```bash
pip install pysdmx[data]
```

In [1]:
from utils import _load_script, streaming_load_save_csv_file, to_vtl_json

streaming_load_save_csv_file(
    golden_copy_original_path="data_files/golden-copy-original.csv",
    output_filename="data_files/golden_copy_changed_10000_sdmx.csv",
    use_sdmx_csv=True, nrows=10000)

Number of lines written: 9553


# Reading the ConceptScheme on SDMX-ML 2.1 using read_sdmx

For this example,
we generated a DataStructure on FMR called LEI_DATA,
with Short URN: DataStructure=MD:LEI_DATA(1.0),
with the required codelists to be used for structural validation on FMR.

This structures is also available at SDMX_Structures/structures.xml file in this project,
or at the MeaningfulData FMR (fmr.meaningfuldata.eu).
Currently the library does not support SDMX-ML 3.0,
so we will read only the ConceptScheme and descendants (available at SDMX_Structures/concepts.xml).

To ensure we are able to validate the data correctly,
we extended the CL_AREA codelist from SDMX
to add a code that was present in the LEI Golden Copy.

The code below reads the ConceptScheme and descendants,
ensure you have installed the xml extra from pysdmx.

```bash
pip install pysdmx[xml]
```

In [2]:
from pysdmx.io import read_sdmx

structures_msg = read_sdmx("SDMX_Structures/concepts.xml")
# We can access the first concept scheme, or look for the short_urn
concept_scheme1 = structures_msg.get_concept_schemes()[0]
concept_scheme2 = structures_msg.get_concept_scheme(
    "ConceptScheme=MD:LEI_CONCEPTS(1.0)")

# Retrieving the Schema from FMR

We may use as well the FMR Webservices to download the Schema from FMR, using the FusionJSON format.

In [3]:
from pprint import pprint
from pysdmx.api.fmr import RegistryClient
from pysdmx.io.format import StructureFormat

client = RegistryClient(
    "https://fmr.meaningfuldata.eu/sdmx/v2", format=StructureFormat.FUSION_JSON
)
# Recommend to use debugger to see the response
schema = client.get_schema(
    "datastructure", agency="MD", id="LEI_DATA", version="1.0"
)
pprint(schema)

Unavailable: Connection error: There was an issue connecting to the targeted registry. The query was `https://fmr.meaningfuldata.eu/sdmx/v2/schema/datastructure/MD/LEI_DATA/1.0`. The error message was: `[Errno 11001] getaddrinfo failed`.

# Using VTL to validate the data with GLEIF data quality checks

The VTL language allows us to perform validations over the data,
with a business friendly syntax. 

For this purpose, at MeaningfulData we have developed a library called vtlengine,
which is able to run VTL scripts over data.

In this example,
we will use a VTL script
that performs validations based on the GLEIF data quality checks
(link) and a custom validation on Subcategory data.


Steps to use VTL from pysdmx:
1. Convert the Schema to a VTL DataStructure
2. Validate the data using VTL
3. Analyse the results

---------(Explanation on VTL validations usefulness,
validating more than one component. Quick overview on the code
using VTL Playground)---------

## Convert the Schema to a VTL DataStructure

This code converts the pysdmx.model Schema and DataStructureDefinition objects into a VTL datastructure,
using MeaningfulData internal format, usable only with vtlengine library.
On pysdmx we will include this method
but it will generate the VTL 2.1 Standard datastructure.
Both options will be usable by the vtlengine library.

In [ ]:
vtl_datastructure = to_vtl_json(schema)
pprint(vtl_datastructure)

## Validate the data using VTL (sample 10000)

---------(Explanation on the code, overview on the VTL run method documentation)---------

## Running the VTL script

In [ ]:
import pandas as pd
from vtlengine import run

script = _load_script("vtl/validations.vtl")
data_df = pd.read_csv("data_files/golden_copy_changed_10000.csv")
datapoints = {"LEI_DATA": data_df}

validations_result = run(script=script, data_structures=vtl_datastructure,
                         datapoints=datapoints)
pprint(validations_result)

### Getting the total number of errors (sample 10000)

In [ ]:
validations_result['errors_count'].data

### Analysing data on Subcategory errors (sample 10000)

In [ ]:
cols_to_analyse = ['CATEGORY', 'SUBCATEGORY', 'errorcode', 'errorlevel']
validations_result['validation.subcategories_errors'].data[cols_to_analyse]

---------(Explanation on Subcategory errors)---------

## Using VTL to perform calculations

--- Explanation on VTL scripts for calculations, overview on the code ---

## Running the VTL script

In [ ]:
script = _load_script("vtl/calculations.vtl")
data_df = pd.read_csv("data_files/golden_copy_changed_10000.csv")
datapoints = {"LEI_DATA": data_df}

calculations_result = run(script=script, data_structures=vtl_datastructure,
                         datapoints=datapoints)
pprint(calculations_result)

In [ ]:
calculations_result['lei_statistics'].data

---------(Explanation on the calculations)---------

# Generate SDMX file with the aggregated data

Generate a PandasDataset from vtlengine output
and use the SDMX-ML 2.1 Data write method from pysdmx.

## Setting up the code

In [ ]:
from pysdmx.io.pd import PandasDataset

data = calculations_result['lei_statistics'].data
structure = "DataStructure=MD:LEI_AGGREGATE_STATISTICS(1.0)"
pd_dataset = PandasDataset(structure=structure, data=data)

### Write the SDMX-ML 2.1 file

In [ ]:
from pysdmx.io.xml.sdmx21.writer.structure_specific import write

output = write([pd_dataset], prettyprint=False)

output

# Reading back the SDMX file using read_sdmx

In [ ]:
from pysdmx.io import read_sdmx

data_msg = read_sdmx(output)
data_msg.data[0].data